In [6]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import HTML

In [ ]:
DATA_URL = 'https://raw.githubusercontent.com/bisdom1/apps/master/assets/owid-covid-data.csv'

In [ ]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}
.app-subtitle a {
    color: #000;
}

.app-subtitle a:hover {
    text-decoration: underline;
}
</style>
""")

In [ ]:
class App:
    
    def __init__(self, df):
        self._df = df
        available_indicators = self._df['location'].unique()
        self._country_dropdown = self._create_indicator_dropdown(available_indicators, 0)       
        self._plot_container = widgets.Output()
        _app_container = widgets.VBox([
            widgets.HBox([self._country_dropdown]),
            self._plot_container
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>\U0001F489 COVID-19 stats</h1>'
                    '<h2 class="app-subtitle">Code adapted from <a href="https://github.com/pbugnion/voila-gallery/blob/master/country-indicators/index.ipynb">this example</a>. Data from <a href="https://ourworldindata.org/coronavirus-source-data">ourworldindata.org</a>.</h2>'
                ), 
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()
        
    @classmethod
    def from_url(cls, url):
        df = pd.read_csv(url, na_values='nan')
        df.date=pd.DatetimeIndex(df.date)
        df.index=df.date
        df['week'] = df.to_period('W').index.strftime("%Y-%m-%d")
        df = df[df['continent'].notna()]
        df = df[df['total_cases'].notna()]
        return cls(df)
        
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    def _create_plot(self, country_indicator):
        fig = px.bar(self._df[self._df.location==country_indicator], x="date", y="new_cases", title="New cases per day in " + country_indicator,template='plotly_white')
        return fig

    def _create_map(self):
        fig = px.scatter_geo(self._df, locations="iso_code", color="continent",
                     hover_name="location", size="total_cases",
                     animation_frame="week",
                     projection="equirectangular",
                     title='Total cases per country, coloured by continent',
                     size_max=50,
                     opacity=0.7,
                     template='plotly_white')
        fig.update_layout(height=600,showlegend=False)
        return fig
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        country_indicator = self._country_dropdown.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            fig = self._create_map()
            fig.show()
            fig = self._create_plot(country_indicator)
            fig.show()

In [ ]:
app = App.from_url(DATA_URL)

app.container